# LT healthcare services

Prepared by [**K.Clemons**](mailto:kimberly.clemons@ext.ec.europa.eu) and [**J.Grazzini**](mailto:jacopo.grazzini@ec.europa.eu) ([_Eurostat_](https://ec.europa.eu/eurostat)).

## Introdution

This notebook illustrates the way to produce *ad-hoc* harmonised data collected from LT national authority. It shows how data can be automatically harmonised, using single-use script-based approachr or a reusable metadata-based approach.

**Table of contents**

* [Setting and checking the environment](#environment).
* [Ad-hoc data ingestion, exploration and processing](#ad-hoc).
* [Metadata-based semi-automated processing](#metadata-processing).
* [Full automation](#automation).
* [Customised processing](#customisation).

## Setting the environment <a id="environment"></a>

In [1]:
PROJECT = 'basic-services'

import os, sys
import functools
import inspect
import json
from pprint import pprint as pprint

import numpy as np
import pandas as pd

In [2]:
FACILITY      = 'HCS'
COUNTRY, LANG = 'Lithuania', 'lt'
CC            = 'LT'
IFILE         = 'Hospitals_2018.xlsx'

In [3]:
THISDIR = os.getcwd()
SRCPATH = os.path.abspath('../../../src/python/pyeufacility/')
DATAPATH = os.path.abspath('../../../data/healthcare/raw/') 
# or       '/Users/<username>/basic-services/data/healthcare/%s' % CC

try:
    print("- src rel. path: \033[1m%s%s\033[0m" % (PROJECT, SRCPATH.split(PROJECT)[1])) 
    print("- data rel. path: \033[1m%s%s\033[0m" % (PROJECT, DATAPATH.split(PROJECT)[1])) 
except:
    print("current path: \033[1m%s\033[0m" % os.getcwd())

- src rel. path: basic-services/src/python/pyeufacility
- data rel. path: basic-services/data/healthcare/raw


## Ad-hoc data ingestion, exploration and processing <a id="ad-hoc"></a>

In [4]:
try:
    import xlrd
except ImportError:
    !{sys.executable} -m pip install xlrd

def adhoc_load(file): # dumb function we aim at reusing later on
    return pd.read_excel(file, header = 2) 

file = os.path.join(DATAPATH, IFILE)
print("\033[4mInput dataset:\033[0m \033[1m%s\033[0m" % file.split(PROJECT)[1])
df_src = adhoc_load(file)

print("\033[4mInput dataset columns:\033[0m \033[1m%s\033[0m" % list(df_src.columns))
# pprint(df_src.dtypes)
df_src.head(5)

Input dataset: /data/healthcare/raw/Hospitals_2018.xlsx
Input dataset columns: ['Code of municipality', 'Municipality', 'ID', 'parent_ID', 'Code of legal entity', 'Subordination: 1-national (MoH), 3-municipality, 8-private, 9-other ministries (not MoH)', 'type_code', 'type_name', 'Level: 1-national, 2-regional, 3-municipality, 4-nursing, 5-other public and specialized, 6-private', 'Name', 'Address', 'Number of beds at the end of the 2018']


,Code of municipality,Municipality,ID,parent_ID,Code of legal entity,"Subordination: 1-national (MoH), 3-municipality, 8-private, 9-other ministries (not MoH)",type_code,type_name,"Level: 1-national, 2-regional, 3-municipality, 4-nursing, 5-other public and specialized, 6-private",Name,Address,Number of beds at the end of the 2018
0,101,Vilnius,1,0,124364561,1,1,general,1,Vilniaus Universiteto ligoninė Santaros klinikos,"Santariškių 2, Vilnius LT-08661",1356
1,101,Vilnius,5,1,302620298,1,1,general,1,Vilniaus Universiteto ligoninės Santaros klini...,"Santariškių 7, Vilnius LT-08406",515
2,101,Vilnius,32,0,124243848,1,1,general,1,Respublikinė Vilniaus universitetinė ligoninė,"Šiltnamių 29, Vilnius",673
3,101,Vilnius,3,0,191744287,1,1,general,1,Vilniaus Universiteto ligoninės Žalgirio klinika,"Žalgirio 117, Vilnius",58
4,101,Vilnius,10,0,124247526,1,19,psychiatry,5,Respublikinė Vilniaus psichiatrijos ligoninė,"Parko g. 21, Vilnius",542


The `adhoc_preparation` method below enables us to clean the dataset, add/rename columns, derive information, *etc...*:

In [5]:
def adhoc_preparation(df):
    df['country'] = COUNTRY
    df['public_private'] = (df['Level: 1-national, 2-regional, 3-municipality, 4-nursing, '
                                '5-other public and specialized, 6-private']
                             .apply(lambda x: 'private' if x==6 else 'public')
                            )

    df.rename(columns = {'ID':                                    'id',
                         'Name':                                  'name',
                         #'Address':                               'address',
                         'Number of beds at the end of the 2018': 'cap_beds',
                         'type_name':                             'facility_type'}, 
               inplace = True, errors = 'ignore') 

    df['place'] = df[['Address', 'country']].apply(', '.join, 1)

    df.drop(columns = ['Subordination: 1-national (MoH), 3-municipality, 8-private, 9-other ministries (not MoH)',
                       'Level: 1-national, 2-regional, 3-municipality, 4-nursing, '
                       '5-other public and specialized, 6-private',
                       'Code of legal entity', 'Code of municipality'],
             inplace = True, errors = 'ignore')

df1 = df_src.copy()
adhoc_preparation(df1)
df1.head(5)

,Municipality,id,parent_ID,type_code,facility_type,name,Address,cap_beds,country,public_private,place
0,Vilnius,1,0,1,general,Vilniaus Universiteto ligoninė Santaros klinikos,"Santariškių 2, Vilnius LT-08661",1356,Lithuania,public,"Santariškių 2, Vilnius LT-08661, Lithuania"
1,Vilnius,5,1,1,general,Vilniaus Universiteto ligoninės Santaros klini...,"Santariškių 7, Vilnius LT-08406",515,Lithuania,public,"Santariškių 7, Vilnius LT-08406, Lithuania"
2,Vilnius,32,0,1,general,Respublikinė Vilniaus universitetinė ligoninė,"Šiltnamių 29, Vilnius",673,Lithuania,public,"Šiltnamių 29, Vilnius, Lithuania"
3,Vilnius,3,0,1,general,Vilniaus Universiteto ligoninės Žalgirio klinika,"Žalgirio 117, Vilnius",58,Lithuania,public,"Žalgirio 117, Vilnius, Lithuania"
4,Vilnius,10,0,19,psychiatry,Respublikinė Vilniaus psichiatrijos ligoninė,"Parko g. 21, Vilnius",542,Lithuania,public,"Parko g. 21, Vilnius, Lithuania"


We can build a geocoder/geolocator using the [`geopy`](https://geopy.readthedocs.io/en/stable/#) package, possibly inserting an API key to the geocoding service if needed (below, `OSM` and `Bing` are considered for the geocoders `gcNominatim` and `gcBing` respectively):

In [6]:
try:
    import geopy
except ImportError:
    !{sys.executable} -m pip install geopy
finally:
    from geopy import geocoders, extra

key = None
try:
    assert key is not None
except:
    gcNominatim = functools.partial(geocoders.Nominatim(user_agent = PROJECT).geocode, 
                                   language = LANG)
else:
    gcBing = geocoders.Bing(user_agent = PROJECT, timeout = 100, api_key = key).geocode

Note some way to circumvent the geocoder limitations (this may not be needed for all geocoders):

In [7]:
print("location: \033[1m%s\033[0m" % df1.iloc[0]['place'])
try:
    location = gcNominatim(df1.iloc[0]['place'])
    assert location is not None
except:
    try:
        location = gcNominatim(df1.iloc[0]['Address'])
        assert location is not None 
    except:
        pass
    else:
        print("lat/lon coordinates: \033[1m(%s,%s)\033[0m" 
              % (location.latitude, location.longitude))
else:
    print("lat/lon coordinates: \033[1m(%s,%s)\033[0m" 
          % (location.latitude, location.longitude))

location: Santariškių 2, Vilnius LT-08661, Lithuania
lat/lon coordinates: (54.75234945,25.277250069302262)


The `adhoc_geolocation` method then applies the geocoder to the newly built `'place'` variable and extract the latitude/longitude coordinates from the geocoder answers. It also handles some negative responses:

In [8]:
def adhoc_location(df, geolocator = gcNominatim):
    df['__coord__'] = df['place'].apply(geolocator)
    if df['__coord__'].isnull().any():
        index = df[df['__coord__'].isnull()].index
        long_address = (df[['name', 'Address', 'country']]
                         .apply(', '.join, 1)
                        ) # of use when geocoding fails
        df.loc[index, '__coord__'] = long_address.loc[index].apply(geolocator)
    df['lat'], df['lon'] = zip(*df['__coord__']
                               .apply(lambda x: (x.latitude, x.longitude) if x != None else (np.nan, np.nan))
                              )
    df.drop(columns = '__coord__', inplace = True, errors = 'ignore') # or keep it for display

adhoc_location(df1)   
df1[['lat', 'lon']].head(5)

GeocoderTimedOut: Service timed out

Finally save the output data:

In [ ]:
OFILE = '%s_geolocated.csv' % CC
df1.to_csv(OFILE)

## Metadata-based semi-automated processing<a id="metadata-processing"></a>

We aim at automating the above operations into a transparent reusable process, while ensuring the potential evolvability of this process (*e.g.*, when data change). For that purpose, we adopt the metadata-based approach implemented throughout the [`pyeudatanat`](https://github.com/eurostat/pyEUDatNat) and [`pyeufacility`](https://github.com/eurostat/pyeufacility) packages.

For a proper import of the packages and setup of the environment, see [the dedicated cells](https://github.com/eurostat/basic-services/blob/master/src/python/notebooks/01_HCS_generic_example_CZ.ipynb#environment) of the [`01_HCS_generic_example_CZ.ipynb` notebook](https://github.com/eurostat/basic-services/blob/master/src/python/notebooks/01_HCS_generic_example_CZ.ipynb).

In [ ]:
try:
    import pyeudatnat
except ImportError:
    !{sys.executable} -m pip install git+https://github.com/eurostat/pyEUDatNat.git
finally:
    from pyeudatnat import misc, io, text, base

try:
    import pyeufacility
except ImportError:
    # !{sys.executable} -m pip install git+https://github.com/eurostat/basic-services.git
    # if you launch it from the notebooks/ directory, try for instance:
    pardir = os.path.abspath(os.path.join(THISDIR, '../'))
    sys.path.insert(0,pardir)
finally:
    from pyeufacility import config, hcs 

Through the LT-specific module `hcs.LThcs` (file `LThcs.py`), we define a series of operations (like the ad-hoc ones above) necessary to 'prepare' the dataset. Those operations are implemented as methods of the `hcs.LThcs.Prepare_data` class (although the accompanying `hcs.LThcs.prepare_data` will be considered, see below):

In [ ]:
Prep_data = hcs.LThcs.Prepare_data
preparator = Prep_data()

pred = lambda o: (inspect.ismethod(o) or inspect.isfunction(o)) and not o.__name__.startswith('__')
print("\033[4mMethods in '%s' class: \033[1m%s\033[0m " 
      % (Prep_data.__name__, [l[0] for l in inspect.getmembers(Prep_data, predicate = pred)]))

* the `split_Adr` function splits any address into its `['street', 'number', 'postcode', 'city']` components:

In [ ]:
split_Adr = preparator.split_Adr
assert callable(split_Adr) is True

print("\033[4mMethod: \033[1m'%s'\033[0m" % split_Adr.__name__)
print(inspect.getsource(split_Adr))

df2 = df_src.copy()

print("\033[4mExample of use of '%s':\033[0m" % split_Adr.__name__)
print(" * input complete address: \033[1m'%s'\033[0m" % df2['Address'].iloc[0])
print(" * output decomposed address: \033[1m'%s'\033[0m" 
      % list(split_Adr(df2['Address'].iloc[0])))

* the `set_adress` method applies the `split_Adr` function on the `'Address'` column, throughout all raws of the input table: 

In [ ]:
set_address = preparator.set_address
assert callable(set_address) is True
print("\033[4mMethod: \033[1m'%s'\033[0m" % set_address.__name__)
print(inspect.getsource(set_address))

set_address(df2)

print("\033[4mExample of use of '%s':\033[0m" % set_address.__name__)
df2[['postcode', 'city', 'street', 'number']].head()

* the `set_pp` method classifies `'Level'` column into `['public', 'private']` types, throughout all raws of the input table:

In [ ]:
set_pp = preparator.set_pp
assert callable(set_pp) is True
print("\033[4mMethod: \033[1m'%s'\033[0m" % set_pp.__name__)
print(inspect.getsource(set_pp))

set_pp(df2)

print("\033[4mExample of use of '%s':\033[0m" % set_pp.__name__)
df2[['public_private']].head()

Besides the basic 'prepare' operations implemented in the `'LThcs'` module, some additional information required for data integration are parsed through the `'LThcs.json'` metadata file:

In [ ]:
METAPATH = os.path.dirname(inspect.getfile(hcs))
print("- Source directory with HCS country-specific metadata: \033[1m'%s'\033[0m" 
      % METAPATH.split(PROJECT)[1])

try:
    meta = os.path.join(METAPATH,'%shcs.json' % CC)
    assert os.path.exists(meta)
except (AssertionError,FileNotFoundError):
    print("! No HCS metadata JSON-file found for '%s'!" % CC)
else:
    print("- Source HCS metadata JSON-file for '%s': \033[1m'%s'\033[0m" 
          % (CC, meta.split(PROJECT)[1]))

In [ ]:
with open(meta, 'r') as fp:
    metadata = io.Json.load(fp)

print("\033[4mCountry-specific metadata for %s:\033[0m" % CC)
pprint(metadata) # print(io.Json.dumps(metadata, indent=2))

The exact one-to-one columns matching from the input raw table to the output harmonised dataset is provided through the `'index'` attribute:

In [ ]:
print("\033[4mInput/output colum matching:\033[0m")
pprint({k:v for (k,v) in metadata['index'].items() if v is not None})

With the two resources above, we can design the LT country-specific ingestion design:
* a `LTFacility` class is derived from the `pyeudatnat.base.BaseDatNat` class using the `pyeufacility.config.facilityFactory` constructor method,
* the `metadata` information (as made available through the `'LThcs.json'` file) is parsed to the `LTFacility` constructor, 
* the 'prepare' operations (as implemented in the `hcs.LThcs.Prepare_data` class) are 'attached' to the `LTFacility` class by overriding the original abstract `BaseDatNat.prepare_data` with the `'hcs.LThcs.prepare_data'` method.

In [ ]:
# build the LT HCS specific class while parsing the metadata
LTFacility = config.facilityFactory(cat = FACILITY, meta = metadata)
# override the prepare_data abstract method
LTFacility.prepare_data = LThcs.prepare_data

pprint(LTFacility.__dict__)

Following, we create an instance `lt` of `LTFacility` to handle these data:

In [ ]:
lt = LTFacility()

print("\033[4mInput data source:\033[0m %s" % lt.file.split(PROJECT)[1])

print("\n\033[4mOutput %s configuration metadata:\033[0m" % FACILITY)
pprint(lt.config.to_dict()) # or simply: print(lt.config)

assert ({k:v for (k,v) in lt.meta.to_dict().items() if v is not None}
        ==  # comparing non empty options
        {k:v for (k,v) in metadata.items() if v is not None})
print("\n\033[4mInput %s country specific metadata:\033[0m" % CC)
pprint(lt.meta.to_dict())

Method `load_data` is used to load the data into the instance dataframe `lt.data`. At this stage, all necessary information to run this operation is available in the `metadata` of the `lt` instance. Besides the file source (`'file'` and `'path'` fields), the option `options['load']` also provides relevant settings for retrieving the (Excel) input data:

In [ ]:
assert ({k:v for (k,v) in lt.options.items() if v not in ({},None)}
        ==  # comparing non empty options
        {k:v for (k,v) in lt.meta['options'].items() if v not in ({},None)})

print("\n\033[4mLoading options:\033[0m %s" % lt.options.get('load')) 
lt.load_data() # try also: lt.load_data(header = 2)
# you can add some keyword arguments necessary to load the data: they will override the default settings

cols = lt.data.columns
print("\n\033[4mInput loaded data columns:\033[0m %s" % list(cols))

Next, we run the `prepare_data` operations that were introduced through the overriding `Prepare_data` class introduced above. This particularly entails the prior creation of the `['number', 'street', 'postcode', 'city']` fields that will be later used for geolocation:

In [ ]:
# print("\033[4mOverriding method: \033[1m'%s'\033[0m" % LThcs.prepare_data.__name__)
# print(inspect.getsource(LThcs.prepare_data))

print("\n\033[4mPre-processing options:\033[0m %s" % lt.options.get('prepare')) 
lt.prepare_data()

print("\n\033[4mUpdated data columns:\033[0m %s" % list(set(lt.data.columns) ^ set(cols)))

Using the previously created fields, the geolocation operation is run through the call to `locate_data`:

In [ ]:
print("\n\033[4mSupported geoservices\033[0m (to be complemented - see geopy):")
pprint(pyeudatnat.geo.CODERS)

print("\n\033[4mGeocoding options:\033[0m %s" % lt.options.get('locate')) 
lt.locate_data() 
# use also: lt.locate_data(place = ['number', 'street', 'postcode', 'city'], gc = {'Bing': 'your-key'})

lt.data.head(5)

Finally, the formatting/harmonisation operated in the `format_data` method consists mainly in basic renaming/casting of existing and new fields/columns:

In [ ]:
lt.format_data()
lt.data.head(5)

## Full automation <a id="automation"></a>

To summarise, the entire data preparation, processing and harmonisation of the input dataset consists in running sequentially the following operations:
```python
# country-specific class definition
LTFacility = config.facilityFactory(cat = 'HCS', meta = metadata)
# facility instance creation
lt = LTFacility()
# data handling, geocoding and formatting
lt.load_data()
lt.prepare_data()
lt.locate_data()
lt.format_data()
```
where processing options/parameters of the various operations above are parsed through the configuration and metadata attributes `meta`, `config` and `options` of the input facility instance, or passed directly to the different methods. 

Overall, the above stepwise process is fully automated once all necessary options/parameters have been informed in the metadata. Namely, it is possible to automatically run the entire processing using the self-contained `harmonise` method: 

In [ ]:
from pyeufacility import harmonise
lt = harmonise.run('HCS', country = "LT", on_disk = True, dest = 'LTtest.csv')

Note that, whenever the geographical coordinates are set in the dataset, it is possible to easily retrieve a `GeoDataFrame` instance from the facility instance using the `to_geodf` method:

In [ ]:
geolt = lt.to_geodf(latlon = ['lat', 'lon'], crs = "EPSG:4326")

geolt[['lat', 'lon', 'geometry']].head(5)

For instance, one can use the geoinformation for basic exploratory visualisation:

In [ ]:
from folium import Map, GeoJson, features
  
vilnius = [54.6872, 25.2797]
zoom = 11
nsample = 10

# gjsonlt = json.dumps(io.Frame.to_geojson(lt.data.iloc[0:(nsample-1),:], latlon = ['lat', 'lon']))
gjsonlt = geolt[~(geolt.lon.isna() | geolt.lat.isna())]
gjsonlt = gjsonlt.iloc[0:(nsample-1),:].to_json()
          
m = Map(location = vilnius, zoom_start = zoom)
GeoJson(gjsonlt, name = 'LT hospital',
    tooltip=features.GeoJsonTooltip(fields=['hospital_name','facility_type','cap_beds'], localize=True)).add_to(m)

m

## Customised processing <a id="customisation"></a>

Using class methods overriding, it is actually possible to customise any of the operations run throughout the entire harmonisation process. Actually, the default implementations made available in the metadata file `LThcs.py` (currently, `Prepare_data` and `prepare_data`) can be overriden themselves. 

This way, any of the loading (`load_data`), pre-processing (`prepare_data`), geolocalisation (`locate_data`) and formatting (`format_data`) operations can be customised/updated. New operations can be added, as listed below: 

In [ ]:
print("\n\033[4mCustomisable operatios:\033[0m \033[1m%s\033[0m" % pyeudatnat.base.PROCESSES)

Operations that are updated and reported in the `LThcs.py` module will be uploaded by the automated `harmonise` process to override the abstract/original methods. Otherwise, overriding can be explicitly performed. 

In the following, we reuse the basic functions used for the [ad-hoc processing](#ad-hoc) throughout the entire workflow. For this purpose a new class `Another_LTFacility` needs to be introduced:

In [ ]:
Another_LTFacility = config.facilityFactory(cat = FACILITY, meta = metadata)

def another_loader(instance): # instance is like 'self'...
    instance.data = adhoc_load(instance.file)
Another_LTFacility.load_data = another_loader

def another_preparator(instance):
    instance.data = instance.data.head(10) # reduce the dataset dimension
    adhoc_preparation(instance.data)
Another_LTFacility.prepare_data = another_preparator

def another_locator(instance):
    adhoc_location(instance.data)
Another_LTFacility.locate_data = another_locator

# pprint(Another_LTFacility.__dict__)

Note that these methods do not use the metadata information (which is instead hardly encoded in the core of the functions). Still, the same stepwise approach is adopted to process the data:

In [ ]:
another_lt = Another_LTFacility()

another_lt.load_data()
another_lt.prepare_data()
another_lt.locate_data()
another_lt.format_data()

another_lt.data.head(5)

More complex functions can be considered, like for the newly defined class `Yet_Another_LTFacility` below (although we also could have overriden the methods of the previously defined class `Another_LTFacility`):

In [ ]:
Yet_Another_LTFacility = config.facilityFactory(cat = FACILITY, meta = metadata)

yet_another_preparator = another_preparator
Yet_Another_LTFacility.prepare_data = yet_another_preparator

def yet_another_locator(instance, delay = True): 
    # remember: we defined earlier the global variable geolocator as:
    #    gcNominatim = functools.partial(geocoders.Nominatim(user_agent = PROJECT).geocode, 
    #                                   language = 'en') # or instance.lang
    if delay is False:
        new_geolocator = gcNominatim
    else: 
        if delay is True:     delay = 1 # delay for requests to be sent to the geoservice
        try:
            assert isinstance(delay, int)
        except: raise IOError("Wrong delay number")
        new_geolocator = extra.rate_limiter.RateLimiter(gcNominatim, min_delay_seconds = delay)
    if 'place' not in instance.data.columns:
        try:
            instance.data['place'] = (
                instance.data[['number', 'street', 'postcode', 'city']].astype(str)
                .apply(lambda s: ', '.join(s), axis=1)
            )
        except: raise IOError("No place available")
    instance.data['lat'], instance.data['lon'] = (
        zip(*instance.data['place']
            .apply(new_geolocator)
            .apply(lambda x: (x.latitude, x.longitude) if x != None else (np.nan, np.nan))
           )
    )
Yet_Another_LTFacility.locate_data = yet_another_locator

And similarly process the data:

In [ ]:
yet_another_lt = Yet_Another_LTFacility()

yet_another_lt.load_data()
yet_another_lt.prepare_data()
yet_another_lt.locate_data()
yet_another_lt.format_data()

yet_another_lt.data.head(5)